In [1]:
import csv
path = "E:\\PycharmProjects\\google-playstore-apps\\Google-Playstore-32K.csv"

In [2]:
import json
import codecs
import math
import re
from statistics import mean 

In [3]:
def vector_normalize(vector):
    vector_length = 0
    for value in vector:
        vector_length += value**2        
    inversed_vector_length = 1/math.sqrt(vector_length)
    normalized_vector = []
    for value in vector:
        normalized_vector.append(value*inversed_vector_length)
    return normalized_vector

In [4]:
with open (path, "r+", encoding="utf-8") as reader:
    csv_reader = csv.reader(reader, delimiter=",", quotechar='"')
    row_count = 0
    columns = []
    data = []
    corrupted_data = []
    for row in csv_reader:
        if row_count == 0:
            columns = row
        else:
            try:
                data.append({
                    columns[0]: row[0],
                    columns[1]: row[1],
                    columns[2]: float(row[2]),
                    columns[3]: int(row[3]),
                    columns[4]: row[4],
                    columns[5]: row[5],
                    columns[6]: float(row[6].replace('$', '')),
                    columns[7]: row[7],
                    columns[8]: row[8],
                    columns[9]: row[9],
                    columns[10]: row[10]
                })
            except:
                corrupted_data.append(row)
#             print(row[0], row[1])
        row_count += 1     

In [5]:
ban = ['FOOD_AND_DRINK', 'TRAVEL_AND_LOCAL', 'SHOPPING', 'LIFESTYLE', 'MUSIC_AND_AUDIO', 'FINANCE',  'EVENTS', 
       'ENTERTAINMENT', 'EDUCATION', 'BOOKS_AND_REFERENCE', 'NEWS_AND_MAGAZINES', 'PHOTOGRAPHY', 'VIDEO_PLAYERS', 
       'ART_AND_DESIGN', 'BUSINESS', 'PRODUCTIVITY', 'COMMUNICATION', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 
       'SOCIAL', 'BEAUTY', 'MAPS_AND_NAVIGATION', 'PERSONALIZATION', 'TOOLS', 'SPORTS', 'AUTO_AND_VEHICLES', 
       'LIBRARIES_AND_DEMO', 'COMICS', 'PARENTING', 'DATING', 'WEATHER', 'MEDICAL']

for row in ban:
    data = list(filter(lambda x: row != x['Category'], data))

In [6]:
genres = []
for game_info in data:
    if game_info['Category'] not in genres:
        genres.append(game_info['Category'])
genres

['GAME_ACTION',
 'GAME_CASUAL',
 'GAME_ROLE_PLAYING',
 'GAME_PUZZLE',
 'GAME_RACING',
 'GAME_ADVENTURE',
 'GAME_ARCADE',
 'GAME_STRATEGY',
 'GAME_SPORTS',
 'GAME_SIMULATION',
 'GAME_MUSIC',
 'GAME_EDUCATIONAL',
 'GAME_WORD',
 'GAME_TRIVIA',
 'GAME_BOARD',
 'GAME_CASINO',
 'GAME_CARD']

In [7]:
data[0]

{'App Name': 'King of Crabs',
 'Category': 'GAME_ACTION',
 'Rating': 3.280254841,
 'Reviews': 785,
 'Installs': '500,000+',
 'Size': '72M',
 'Price': 0.0,
 'Content Rating': 'Everyone 10+',
 'Last Updated': 'March 7, 2019',
 'Minimum Version': '4.4 and up',
 'Latest Version': '1.0.8'}

In [8]:
installs = []
for game_info in data:
    tmp = int(game_info['Installs'].replace(',', '').replace('+', ''))
    if tmp not in installs:
        installs.append(tmp)
installs.sort(key = lambda x: x)

In [9]:
installs_new = []
i = len(installs)

while i > 2:
    installs_new.append([installs[i-2], installs[i-1]])
    i -= 1

installs_ii = installs[1:-1]

In [10]:
paid_games = list(filter(lambda x: x['Price'] != 0, data))
free_games = list(filter(lambda x: x['Price'] == 0, data))

with open('results.csv', 'a+') as csv_writer:
    csv_writer.write('От;До;Жанр;Средняя прибыль;Средняя цена;Количество платных игр;Количество бесплатных игр' + '\n')

    for installs_count in installs_ii:
        border = list(filter(lambda x: x[0] == installs_count, installs_new))[0]

        games_by_owners = list(filter(lambda x: float(x['Installs'].replace(',', '').replace('+', '')) == installs_count, paid_games))
        free_games_by_owners = list(filter(lambda x: float(x['Installs'].replace(',', '').replace('+', '')) == installs_count, free_games))

        print(f"\n{border}")
        for genre in genres:
            games_by_genres = list(filter(lambda x: genre == x['Category'], games_by_owners))
            free_games_by_genres = list(filter(lambda x: genre == x['Category'], free_games_by_owners))

            if(len(games_by_genres) != 0):
                mean_income = sum(list(map(lambda y: y['Price'] * mean(border), games_by_genres)))/len(games_by_genres)
                mean_price = sum(list(map(lambda y: y['Price'], games_by_genres)))/len(games_by_genres)
                print(f'{genre} - {round(mean_income)} - {round(mean_price, 2)} - {len(games_by_genres)} - {len(free_games_by_genres)}')
                
                csv_writer.write(f"{border[0]};{border[1]};{genre};{str(round(mean_income, 2)).replace('.', ',')};{str(round(mean_price, 2)).replace('.', ',')};{len(games_by_genres)};{len(free_games_by_genres)}\n")
            else:
                csv_writer.write(f"{border[0]};{border[1]};{genre};{0};{0};{len(games_by_genres)};{len(free_games_by_genres)}\n")


[1, 5]
GAME_ADVENTURE - 3 - 0.99 - 1 - 0
GAME_ARCADE - 3 - 0.99 - 1 - 0
GAME_EDUCATIONAL - 3 - 0.99 - 5 - 0

[5, 10]
GAME_ACTION - 15 - 1.99 - 1 - 0
GAME_PUZZLE - 7 - 0.99 - 1 - 2
GAME_CARD - 37 - 4.89 - 1 - 0

[10, 50]
GAME_ACTION - 75 - 2.49 - 1 - 1
GAME_PUZZLE - 70 - 2.32 - 3 - 4
GAME_RACING - 30 - 0.99 - 1 - 0
GAME_ADVENTURE - 100 - 3.32 - 3 - 0
GAME_ARCADE - 60 - 1.99 - 2 - 6
GAME_STRATEGY - 120 - 3.99 - 2 - 1
GAME_WORD - 60 - 1.99 - 1 - 0

[50, 100]
GAME_ACTION - 74 - 0.99 - 2 - 2
GAME_CASUAL - 74 - 0.99 - 1 - 2
GAME_ROLE_PLAYING - 74 - 0.99 - 1 - 0
GAME_PUZZLE - 121 - 1.61 - 4 - 0
GAME_ADVENTURE - 318 - 4.24 - 2 - 2
GAME_ARCADE - 187 - 2.49 - 3 - 3
GAME_STRATEGY - 249 - 3.32 - 3 - 1
GAME_SIMULATION - 112 - 1.49 - 1 - 1
GAME_EDUCATIONAL - 74 - 0.99 - 1 - 3

[100, 500]
GAME_ACTION - 1610 - 5.36 - 4 - 6
GAME_CASUAL - 597 - 1.99 - 2 - 9
GAME_ROLE_PLAYING - 597 - 1.99 - 1 - 0
GAME_PUZZLE - 1047 - 3.49 - 8 - 14
GAME_ADVENTURE - 2097 - 6.99 - 1 - 1
GAME_ARCADE - 772 - 2.57 - 6 - 12
GA

In [11]:
for genre in genres:
    tmp = list(filter(lambda x: genre == x['Category'], data))
    tmp.sort(key = lambda x: float(x['Installs'].replace(',', '').replace('+', '')), reverse=True)
    print(f"{genre}:")
    for i in range(10):
        print(f"№{i+1}: {tmp[i]['App Name']}")
    print("\n")  

GAME_ACTION:
№1: Temple Run 2
№2: Shadow Fight 2
№3: Kick the Buddy
№4: Helix Jump
№5: PUBG MOBILE
№6: slither.io
№7: Sniper 3D Gun Shooter: Free Elite Shooting Games
№8: Modern Combat 5: eSports FPS
№9: Garena Free Fire
№10: Crossy Road


GAME_CASUAL:
№1: Candy Crush Saga
№2: Pou
№3: My Talking Tom
№4: Candy Crush Soda Saga
№5: Angry Birds 2
№6: My Talking Angela
№7: Minion Rush: Despicable Me Official Game
№8: Talking Tom Cat
№9: Farm Heroes Saga
№10: Hay Day


GAME_ROLE_PLAYING:
№1: Summoners War
№2: MARVEL Future Fight
№3: Angry Birds Epic RPG
№4: Empires & Puzzles: RPG Quest
№5: Star Wars�: Galaxy of Heroes
№6: Game of Sultans
№7: Idle Heroes
№8: Hustle Castle: Fantasy Kingdom
№9: SoulCraft - Action RPG (free)
№10: Eternium


GAME_PUZZLE:
№1: Where's My Water? Free
№2: Flow Free
№3: Roll the Ball� - slide puzzle
№4: Where's My Water? 2
№5: Cut the Rope FULL FREE
№6: Cut the Rope 2
№7: Shoot Bubble Deluxe
№8: Candy Crush Jelly Saga
№9: Block Puzzle Jewel
№10: Love Balls


GAME_RACI